In [0]:
import pyspark
from pyspark.sql import SparkSession

In [0]:
spark=SparkSession.builder.appName('lr_ecom_example').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
data=spark.read.csv('/FileStore/tables/Ecommerce_Customers.csv',inferSchema=True, header=True)

In [0]:
data.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [0]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
 Email| Address| Avatar|Avg Session Length| Time on App| Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
mstephenson@ferna...|835 Frank TunnelW...| Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616| 4.0826206329529615| 587.9510539684005|
 hduke@hotmail.com|4547 Archer Commo...| DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744| 2.66403418213262| 392.2049334443264|
 pallen@yahoo.com|24645 Valerie Uni...| Bisque|33.000914755642675|11.330278057777512|37.110597442120856| 4.104543202376424| 487.54750486747207|
riverarebecca@gma...|1414 David Throug...| SaddleBrown| 34.30555662975554|13.717513665142507| 36.72128267790313| 3.120178782748092| 581.8523440352177|
mstephens@davidso...|14023 Rodriguez P...|MediumAquaMarine| 33.33067252364639|12.795188551078114| 37.53665330059473| 4.446308318351434| 599.4060920457634|
alvareznancy@luca...|645 Martha Park A...| FloralWhite|33.871037879341976|12.026925339755056| 34.47687762925054| 5.493507201364199| 637.102447915074|
katherine20@yahoo...|68388 Reyes Light...| DarkSlateBlue| 32.02159550138701|11.366348309710526| 36.68377615286961| 4.685017246570912| 521.5721747578274|
 awatkins@yahoo.com|Unit 6538 Box 898...| Aqua|32.739142938380326| 12.35195897300293| 37.37335885854755| 4.4342734348999375| 549.9041461052942|
vchurch@walter-ma...|860 Lee KeyWest D...| Salmon| 33.98777289568564|13.386235275676436|37.534497341555735| 3.2734335777477144| 570.2004089636196|
 bonnie69@lin.biz|PSC 2734, Box 525...| Brown|31.936548618448917|11.814128294972196| 37.14516822352819| 3.202806071553459| 427.1993848953282|
andrew06@peterson...|26104 Alexander G...| Tomato|33.992572774953736|13.338975447662113| 37.22580613162114| 2.482607770510596| 492.6060127179966|
ryanwerner@freema...|Unit 2413 Box 034...| Tomato| 33.87936082480498|11.584782999535266| 37.08792607098381| 3.71320920294043| 522.3374046069357|
 knelson@gmail.com|6705 Miller Orcha...| RoyalBlue|29.532428967057943|10.961298400154098| 37.42021557502538| 4.046423164299585| 408.6403510726275|
wrightpeter@yahoo...|05302 Dunlap Ferr...| Bisque| 33.19033404372265|12.959226091609382|36.144666700041924| 3.918541839158999| 573.4158673313865|
taylormason@gmail...|7773 Powell Sprin...| DarkBlue|32.387975853153876|13.148725692056516| 36.61995708279922| 2.494543646659249| 470.4527333009554|
 jstark@anderson.com|49558 Ramirez Roa...| Peru|30.737720372628182|12.636606052000127|36.213763093698624| 3.3578468423262944| 461.7807421962299|
 wjennings@gmail.com|6362 Wilson Mount...| PowderBlue| 32.12538689728784|11.733861690857394| 34.8940927514398| 3.1361327164897803| 457.84769594494855|
rebecca45@hale-ba...|8982 Burton RowWi...| OliveDrab|32.338899323067196|12.013194694014402| 38.38513659413844| 2.420806160901484| 407.70454754954415|
alejandro75@hotma...|64475 Andre Club ...| Cyan|32.187812045932155| 14.7153875441565| 38.24411459434352| 1.516575580831944| 452.3156754800354|
samuel46@love-wes...|544 Alexander Hei...| LightSeaGreen| 32.61785606282345|13.989592555825254|37.190503800397956| 4.064548550437977| 605.061038804892|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
only showing top 20 rows

In [0]:
data.head(1)[0]

Out[10]: Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [0]:
for i in data.head(1)[0]:
  print(i)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
data.columns

Out[16]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
assembler=VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership']
                          , outputCol='features')

In [0]:
output=assembler.transform(data)

In [0]:
output.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
output.head(1)

Out[29]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [0]:
output.select('features').show()

+--------------------+
 features|
+--------------------+
[34.4972677251122...|
[31.9262720263601...|
[33.0009147556426...|
[34.3055566297555...|
[33.3306725236463...|
[33.8710378793419...|
[32.0215955013870...|
[32.7391429383803...|
[33.9877728956856...|
[31.9365486184489...|
[33.9925727749537...|
[33.8793608248049...|
[29.5324289670579...|
[33.1903340437226...|
[32.3879758531538...|
[30.7377203726281...|
[32.1253868972878...|
[32.3388993230671...|
[32.1878120459321...|
[32.6178560628234...|
+--------------------+
only showing top 20 rows

In [0]:
final_data=output.select('features','Yearly Amount Spent')

In [0]:
final_data.show()

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
[33.0009147556426...| 487.54750486747207|
[34.3055566297555...| 581.8523440352177|
[33.3306725236463...| 599.4060920457634|
[33.8710378793419...| 637.102447915074|
[32.0215955013870...| 521.5721747578274|
[32.7391429383803...| 549.9041461052942|
[33.9877728956856...| 570.2004089636196|
[31.9365486184489...| 427.1993848953282|
[33.9925727749537...| 492.6060127179966|
[33.8793608248049...| 522.3374046069357|
[29.5324289670579...| 408.6403510726275|
[33.1903340437226...| 573.4158673313865|
[32.3879758531538...| 470.4527333009554|
[30.7377203726281...| 461.7807421962299|
[32.1253868972878...| 457.84769594494855|
[32.3388993230671...| 407.70454754954415|
[32.1878120459321...| 452.3156754800354|
[32.6178560628234...| 605.061038804892|
+--------------------+-------------------+
only showing top 20 rows

In [0]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [0]:
train_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 343|
 mean| 496.41525226335926|
 stddev| 78.91958199345412|
 min| 256.67058229005585|
 max| 744.2218671047146|
+-------+-------------------+

In [0]:
test_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 157|
 mean| 505.6470547959434|
 stddev| 80.05881961905293|
 min| 275.9184206503857|
 max| 765.5184619388373|
+-------+-------------------+

In [0]:
lr=LinearRegression(labelCol='Yearly Amount Spent')

In [0]:
lr_model=lr.fit(train_data)

In [0]:
test_results=lr_model.evaluate(test_data)

In [0]:
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
 -4.671738754718888|
-14.329968140986637|
 3.5794524504079845|
-0.7862047408177659|
 -9.88968080790039|
 -5.243857751377902|
 -6.938501149449053|
 -9.248887779136282|
-2.5293870497795297|
 -2.243492677357949|
-2.4369230831829896|
-12.848607503358835|
 -3.992377021948812|
 -9.465875378272301|
-17.786619263089108|
 -2.733836915238669|
-14.541289388553196|
 3.1234661549550538|
 15.630884932316917|
 -3.29857233134544|
+-------------------+
only showing top 20 rows

In [0]:
test_results.rootMeanSquaredError

Out[41]: 9.818553204013936

In [0]:
test_results.r2

Out[42]: 0.9848625861534926

In [0]:
#some unlabeled_data with features, let's predict the ouput

In [0]:
unlabeled_data=test_data.select('features')

In [0]:
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
[30.8364326747734...|
[31.0662181616375...|
[31.3584771924370...|
[31.3895854806643...|
[31.4474464941278...|
[31.5171218025062...|
[31.5257524169682...|
[31.5261978982398...|
[31.5761319713222...|
[31.6610498227460...|
[31.7216523605090...|
[31.8093003166791...|
[31.8186165667690...|
[31.8279790554652...|
[31.9048571310136...|
[31.9120759292006...|
[31.9365486184489...|
[32.0215955013870...|
[32.0609143984100...|
[32.0775900443291...|
+--------------------+
only showing top 20 rows

In [0]:
predictions=lr_model.transform(unlabeled_data)

In [0]:
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[30.8364326747734...| 472.1736391817085|
[31.0662181616375...| 463.263261348661|
[31.3584771924370...| 491.5964979990674|
[31.3895854806643...|410.85581580080066|
[31.4474464941278...| 428.4924229031244|
[31.5171218025062...| 281.1622784017636|
[31.5257524169682...|450.90412795933094|
[31.5261978982398...| 418.3434139714741|
[31.5761319713222...| 543.7559710391079|
[31.6610498227460...| 418.6018462572588|
[31.7216523605090...|350.21384971505563|
[31.8093003166791...| 549.6205068662|
[31.8186165667690...|450.41105039208446|
[31.8279790554652...| 449.4686229252138|
[31.9048571310136...|491.73647668590525|
[31.9120759292006...| 390.2685532209464|
[31.9365486184489...| 441.7406742838814|
[32.0215955013870...| 518.4487086028723|
[32.0609143984100...| 611.9724337806981|
[32.0775900443291...| 404.3317075504515|
+--------------------+------------------+
only showing top 20 rows